In [51]:
%load_ext autoreload
%autoreload 2
from models import *
from names import *
import seaborn as sns
import matplotlib.pyplot as plt
import pdfkit
from weasyprint import HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
results_df = aggregate_results_from_subfolders(".")

In [53]:
results_df.sort_values(by="difference of ACE", ascending=True, inplace=True)
# results_df.sort_values(by="pre_r_squared test", ascending=False, inplace=True)

results_df.to_csv("results.csv", index=False)
print(f"{len(results_df)} records")

63 records


In [54]:
# results_digest = results_df[
#     [
#         "precovid ACE",
#         "postcovid ACE",
#         "difference of ACE",
#         "outcome",
#         "treatment",
#     ]
# ].copy()
results_digest = results_df[
    [
        "difference of ACE",
        "outcome",
        "treatment",
    ]
].copy()
results_digest["outcome description"] = results_digest["outcome"].apply(
    lambda x: full_dictionary[x]
)
results_digest["treatment description"] = results_df["treatment"].apply(
    lambda x: full_dictionary[x]
)
result_digest = results_digest[
    ["treatment description", "outcome description", "difference of ACE"]
]

In [55]:
len(results_digest["outcome"].unique())

7

In [56]:
len(results_digest["treatment"].unique())

14

In [57]:
df_wide = results_digest.pivot(
    index="treatment description",
    columns="outcome description",
    values="difference of ACE",
)
df_wide

outcome description,gad2_score,pam,phq2_score,phq4_score,social_level,sse_score,stress
treatment description,,,,,,,
At home,-0.08430,0.6390,-0.11600,-0.16900,NaN,-0.0158,-0.1100
Distance traveled,NaN,NaN,NaN,NaN,-0.1610,NaN,NaN
Duration unlocked phone in social loc.,0.19100,-0.8100,0.06110,0.49800,NaN,-0.7370,0.1520
Exercise,-0.08610,NaN,NaN,-0.01030,0.1190,0.3450,-0.0590
Frequency of unlocked phone in social loc.,-0.14400,NaN,0.01700,-0.01320,-0.0495,0.1830,-0.1000
Motion at social locations,-0.11800,NaN,NaN,NaN,0.0375,NaN,NaN
Sports,-0.08080,0.0852,-0.20700,-0.33700,0.0783,0.5930,-0.0288
Studying,-0.00272,-0.6410,NaN,-0.15700,0.0987,-0.3930,NaN
Time in social loc.,0.10600,-0.2360,-0.22900,0.00291,0.1240,-1.2000,0.1180


In [58]:
import pdfkit


def color_gradient(val):
    # Normalize the value to the range [0, 1]
    norm = (val - df_wide.min().min()) / (
        df_wide.max().max() - df_wide.min().min()
    )
    # Use a colormap to get the color
    cmap = plt.get_cmap("coolwarm")
    # Get the color as a tuple (r, g, b, a)
    color = cmap(norm)
    return (
        f"background-color: rgba({int(color[0]*255)}, {int(color[1]*255)},"
        f"{int(color[2]*255)}, {color[3]}); color: black; "
        "font-family: Arial; font-size: 12px;font-weight: bold;"
    )


df_wide.drop(columns="pam", inplace=True)
styled_df = df_wide.style.map(color_gradient)
styled_df = styled_df.format("{:.2e}", na_rep="")

# Add borders to the cells
styled_df = styled_df.set_table_styles(
    [{"selector": "th, td", "props": [("border", "1px solid black")]}]
)
# Convert the styled DataFrame to HTML
html = styled_df.to_html()

# Save the HTML to a file (optional, for debugging)
with open("styled_dataframe.html", "w") as file:
    file.write(html)

options = {
    "disable-smart-shrinking": "",
}

pdfkit.from_file(
    "styled_dataframe.html", "styled_dataframe.pdf", options=options
)
# html_object = HTML("styled_dataframe.html")
# html_object.write_pdf("output_weasy.pdf", zoom=1)

True